In [ ]:
import concurrent.futures
from collections import Counter
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk import sent_tokenize, word_tokenize
from itertools import chain
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
encoder = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-10-h-128-a-2/2", trainable=False)

## Global variables

In [ ]:
TRAIN_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"
TEST_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
OUTPUT_PATH = '/kaggle/working/'

# Model parameters:
# as this is a simple model we use small vocabulary size and truncate to 400 words
NUM_EPOCHS = 100
VOCABULARY_SIZE = 10000
EMBEDDING_DIM = 120
MAX_LENGTH = 4000
TRUNCATION_TYPE = 'post'
PADDING_TYPE = 'pre'
OOV_TOKEN = '<OOV>'
TRAIN_SIZE = .75
PATIENCE = 2

In [ ]:
# that are the 7 topics of the essays and the corresponding leading tf idf terms
venus=['venus']
mars=['mars','face','alien','aliens','landform','martian','mesa']
cars=['accidents','self','smart','cars','car','driverless','driveless','usage','sensors','drive','driver','transportation','vehicle','vehicles','driving']
electoral=['vote','electoral','president','electors', 'state', 'senator','collage','ellection']
technology=['happy','huang','muscles','mona','lisa','computer','software','technology','emotions','facial','emotion','student','students','kids','teachers','feeling','teacher','feel','expressions']
luke=['luke','cowboy', 'seagoing','seagoing','cowboys','program','animals']
pollution=['air','pollution','smog','emissions','environment','city','paris','cities']
topic_words = venus + mars + cars + electoral +technology + luke + pollution

## Loading data

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH)

## Finding all words similar to known topic words

First I will find all the different words that appear at least 3 times across all texts.

In [ ]:
all_text = ' '.join(df_train['full_text'])
# Tokenize the text into words, convert to lowercase, and extract unique words
words = re.findall(r'\b\w+\b', all_text.lower())
word_counts = Counter(words)
# Filter words that occur at least three times
words_used_at_least_three_times = [word for word, count in word_counts.items() if count >= 3]
print(f'Words used at least three times: {len(words_used_at_least_three_times)}')
unique_words = set(words_used_at_least_three_times)
vocab = list(unique_words)
vocab = sorted(vocab, key=lambda word: word_counts[word], reverse=True)

Now I create embeddings for known topic words and for all frequent words. And I compare the vectors to find words that are similar to topic words.

After trying different thresholds I decided to choose 0.984 of similarity. 

In [ ]:
# as the vocabulary is more than 20 K we need to calculate embeddings in batches
def get_bert_embedding(word, preprocessor, encoder):
    text_input = tf.constant([word])
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]  # [batch_size, 128]
    return pooled_output.numpy()

# Get embeddings for topic words
topic_embeddings = {word: get_bert_embedding(word, preprocessor, encoder) for word in topic_words}

# Get embeddings for the vocabulary
def process_batch(words):
    embeddings = {}
    for word in words:
        embeddings[word] = get_bert_embedding(word, preprocessor, encoder)
    return embeddings

# Split the vocabulary into batches
batch_size = 1000
ending=len(vocab)
#for testing
#ending=500
batches = [vocab[i:i+batch_size] for i in range(0, ending, batch_size)]

# Process batches in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_batch, batches)

# Combine embeddings from all batches
vocab_embeddings = {}
batch_count = 0
for result in results:
    batch_count += 1
    print(f"Processed batch {batch_count} out of {len(batches)}")
    vocab_embeddings.update(result)
print (2)

# Function to find similar words based on cosine similarity
def get_similar_words(topic_embeddings, vocab_embeddings, threshold=0.8):
    similar_words = set()
    for topic_word, topic_emb in topic_embeddings.items():
        for vocab_word, vocab_emb in vocab_embeddings.items():
            if topic_word != vocab_word:
                similarity = cosine_similarity(topic_emb, vocab_emb)[0][0]
                if similarity > threshold:
                    similar_words.add(vocab_word)
    return similar_words

# Find similar words (for testing different thresholds)
#for tr in np.arange(0.98,0.99,0.002):
#    related_words = get_similar_words(topic_embeddings, vocab_embeddings, threshold=tr)
#    print(f'{tr} Related Words: {len(related_words)}')
#    print (related_words)

In [ ]:
related_words = get_similar_words(topic_embeddings, vocab_embeddings, threshold=0.984)
# Combine topic words and related words
all_exclude_words = set(topic_words).union(related_words)

In the column with text in the df I change all topic words and all words similar to those for word "topic". The objective is to concentrate on the structure of the essays rather then on topics and make it easier for RNN to train the model.

In [ ]:
def exclude_words(text, words):
    pattern = re.compile(r'\b(' + '|'.join(words) + r')\b', re.IGNORECASE)
    return pattern.sub("topic", text)

In [ ]:
df_train['txt_wth_topic']=df_train['full_text'].apply(lambda x : exclude_words(x,all_exclude_words))

## RNN Model to predict the score

In [ ]:
print ("running the model")

In [ ]:
# shuffle the data
df = df_train.sample(frac=1).reset_index(drop=True)
# Select the columns with the features
df_data_x = df[['txt_wth_topic']]
# Select the column with the label:
df_data_y = df['score'].astype(int)

# Split the samples:
x_train, x_test, y_train, y_test = train_test_split(df_data_x,
                                                    df_data_y,
                                                    train_size=TRAIN_SIZE,
                                                    random_state=42)

# create tokenizer
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(x_train['txt_wth_topic'])  
train_sequences = tokenizer.texts_to_sequences(x_train['txt_wth_topic'])

# padding / truncating
train_padded = pad_sequences(train_sequences,
                             maxlen=MAX_LENGTH,
                             padding=PADDING_TYPE,
                             truncating=TRUNCATION_TYPE)

validation_sequences = tokenizer.texts_to_sequences(x_test['txt_wth_topic'])
validation_padded = pad_sequences(validation_sequences,
                                  maxlen=MAX_LENGTH,
                                  padding=PADDING_TYPE,
                                  truncating=TRUNCATION_TYPE)

# encoding
encoder = LabelEncoder()
encoder.fit(y_train)    
encoded_y_train = to_categorical(encoder.transform(y_train))
encoded_y_test = to_categorical(encoder.transform(y_test))
num_classes = len(np.unique(y_train))

In [ ]:
model = tf.keras.Sequential([
tf.keras.layers.Embedding(VOCABULARY_SIZE, EMBEDDING_DIM),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.LSTM(128, return_sequences=True),
tf.keras.layers.LSTM(64),
tf.keras.layers.Dense(256, activation="relu"),
tf.keras.layers.Dense(128, activation="relu"),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(1, activation="linear")
]) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mse"])

# Build the model by providing the input shape
model.build(input_shape=(None, MAX_LENGTH))

# summary and callback
model.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE)

# training
history = model.fit(train_padded,
                    y_train,
                    epochs=NUM_EPOCHS,
                    validation_data=(validation_padded, y_test),
                    callbacks=[callback]
                    ) 

predictotron = model.predict(validation_padded) 

## Printing

In [ ]:
def printing (chosen_history):
    model_history = pd.DataFrame(chosen_history.history)
    model_history['epoch'] = chosen_history.epoch

    fig, ax = plt.subplots(1, figsize=(8, 6))
    num_epochs = model_history.shape[0]

    ax.plot(np.arange(0, num_epochs), model_history["mse"], label="Training mse")
    ax.plot(np.arange(0, num_epochs), model_history["val_mse"], label="Validation mse")
    ax.legend()

    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(1, figsize=(8, 6))
    num_epochs = model_history.shape[0]

    ax.plot(np.arange(0, num_epochs), model_history["loss"], label="Training loss")
    ax.plot(np.arange(0, num_epochs), model_history["val_loss"], label="Validation loss")
    ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
printing(history)

## Test

In [ ]:
df_test = pd.read_csv(TEST_DATA_PATH)
df_test['txt_wth_topic']=df_test['full_text'].apply(lambda x : exclude_words(x,all_exclude_words))


## Predict

In [ ]:
def predict (df_to_predict, chosen_model):
    df_sequences = tokenizer.texts_to_sequences(df_to_predict['txt_wth_topic'])
    df_sequences = pad_sequences(df_sequences,
                                     maxlen=MAX_LENGTH,
                                     padding=PADDING_TYPE,
                                     truncating=TRUNCATION_TYPE)

    prediction = chosen_model.predict(df_sequences)
    prediction=prediction.clip(1, 6).round()
    prediction=prediction.astype(int)
    return prediction

In [ ]:
prediction_test = predict(df_test,model)
print(prediction_test[:,0])
predictions_df = pd.DataFrame({'essay_id': df_test['essay_id'],
                               'score': prediction_test[:,0]})
print(predictions_df)

predictions_df.to_csv(os.path.join(OUTPUT_PATH, "submission.csv"), index=False)